Hypothesis of different types of weather being from normal distribution

Total flights per day and impact on taxi times. taxi times impact on delays. avgf taxi time for major hubs ( NY, ATL, DAL, LAX)

Average delay based on time of year, month of year, day of week. 

Do planes have higher average air speed when leaving after a delay to make up time?

How much does time of departure impact potential delays

NAS delay is national air system delay. could one-hot encode variables such as this and use average value to determine an amount for generic delay

carrier delay occurs roughly half the time
arrival negative delays time avg time is 40 minutes.  
approximately 35% of flights are delayed negatively( leaving later than intended) 5475609/10140132 = 0.3506

average negative delay is 40 min



scaling needed for variables

weather api research 
last 10 days december 2018, 2019, first 2 weeks of janurary 2018, 2019


In [1]:
# SQL query used to find relevant dates to compare with weather data

'''

SELECT origin, COUNT(origin) AS flights FROM flights 
WHERE fl_date LIKE '%-12-2%' 
OR fl_date LIKE '%-12-3%' 
OR fl_date LIKE '%-01-0%'
GROUP BY origin
ORDER BY flights DESC
LIMIT 10;

'''

# Selected list of top 10 airports with most origin flights in the last 11 days of december 
# and the first 10 days in january


'''

10 rows returned
origin
text
flights
bigint
1	ORD	42120
2	ATL	41504
3	DFW	32710
4	DEN	30838
5	LAX	29128
6	CLT	27895
7	PHX	21197
8	SEA	21005
9	IAH	20263
10	SFO	20120

'''

# Will manually pull lat. and long. data for all of these airports to run through weather api

'\n\n10 rows returned\norigin\ntext\nflights\nbigint\n1\tORD\t42120\n2\tATL\t41504\n3\tDFW\t32710\n4\tDEN\t30838\n5\tLAX\t29128\n6\tCLT\t27895\n7\tPHX\t21197\n8\tSEA\t21005\n9\tIAH\t20263\n10\tSFO\t20120\n\n'

In [2]:
airport_coords = {'ORD':'41.978611,-87.904724',
                  'ATL':'33.640411,-84.419853',
                  'DFW':'32.779167,-96.808891',
                  'DEN':'39.849312,-104.67382',
                  'LAX':'33.942791,-118.41004',
                  'CLT':'35.213890,-80.943054',
                  'PHX':'33.448376,-112.07403',
                  'SEA':'47.443546,-122.30165',
                  'IAH':'29.980498,-95.339657',
                  'SFO':'37.615223,-122.38997'
                 }

In [3]:
# found dataset that made coordinates irrelevant, using weather station Airport Code

In [4]:
airport_codes = ['KORD', 'KATL', 'KDFW', 'KDEN', 'KLAX', 'KCLT', 'KPHX', 'KSEA', 'KIAH', 'KSFO']

In [6]:
# import requests as re
# import config as cfg

In [7]:
# ambee_key = cfg.AMBEE_KEY

In [8]:
import json

In [9]:
# requests no longer needed as data was found in a dataset




# def get_weather(lat,long):
    

#     url = "https://api.ambeedata.com/weather/history/by-lat-lng"
#     querystring = {"lat":lat,"lng":long,"from":"2019-01-01 00:00:00","to":"2019-01-06 00:00:00"}
#     headers = {
#         'x-api-key': ambee_key,
#         'Content-type': "application/json"
#         }
#     response = re.request("GET", url, headers=headers, params=querystring)
#     return response.json()

In [11]:
# data = get_weather(ORD_lat,ORD_long)

In [12]:
# import pprint

In [14]:
# reading csv fround on kaggle of weather events in the united states from 2016-2020

In [15]:
import pandas as pd

In [66]:
# importing weather events dataset downloaded from kaggle

In [16]:
csv = pd.read_csv('./WeatherEvents_Jan2016-Dec2020.csv')

In [17]:
type(csv['LocationLng'][0])

numpy.float64

In [18]:
csv[csv['City'] == 'Chicago'].AirportCode.value_counts()

KMDW    2894
Name: AirportCode, dtype: int64

In [19]:
# finding only those entries which match list of most busy airports

common_airports = csv[csv['AirportCode'].isin(airport_codes)]
common_airports.head()

,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode
386388,W-386466,Snow,Light,2016-01-01 11:51:00,2016-01-01 12:51:00,US/Central,KORD,41.9875,-87.9319,Bensenville,DuPage,IL,60106.0
386389,W-386467,Snow,Moderate,2016-01-01 12:51:00,2016-01-01 13:51:00,US/Central,KORD,41.9875,-87.9319,Bensenville,DuPage,IL,60106.0
386390,W-386468,Snow,Light,2016-01-03 21:51:00,2016-01-03 22:51:00,US/Central,KORD,41.9875,-87.9319,Bensenville,DuPage,IL,60106.0
386391,W-386469,Snow,Light,2016-01-03 23:12:00,2016-01-04 04:51:00,US/Central,KORD,41.9875,-87.9319,Bensenville,DuPage,IL,60106.0
386392,W-386470,Snow,Light,2016-01-04 14:51:00,2016-01-04 18:51:00,US/Central,KORD,41.9875,-87.9319,Bensenville,DuPage,IL,60106.0


In [67]:
# retrieving only the date portion of information start time

In [20]:
common_airports['WeatherDate'] = common_airports['StartTime(UTC)'].apply(lambda x: pd.to_datetime(x).date())

<ipython-input-20-4836ab25a1ca>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  common_airports['WeatherDate'] = common_airports['StartTime(UTC)'].apply(lambda x: pd.to_datetime(x).date())


In [68]:
# retrieving only the time portion of information start time

In [21]:
import datetime
common_airports['Time'] = common_airports['StartTime(UTC)'].apply(lambda x: pd.to_datetime(x).time())

<ipython-input-21-29b693c015ef>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  common_airports['Time'] = common_airports['StartTime(UTC)'].apply(lambda x: pd.to_datetime(x).time())


In [23]:
common_airports.dtypes

EventId            object
Type               object
Severity           object
StartTime(UTC)     object
EndTime(UTC)       object
TimeZone           object
AirportCode        object
LocationLat       float64
LocationLng       float64
City               object
County             object
State              object
ZipCode           float64
WeatherDate        object
Time               object
dtype: object

In [24]:
common_airports.head()

,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode,WeatherDate,Time
386388,W-386466,Snow,Light,2016-01-01 11:51:00,2016-01-01 12:51:00,US/Central,KORD,41.9875,-87.9319,Bensenville,DuPage,IL,60106.0,2016-01-01,11:51:00
386389,W-386467,Snow,Moderate,2016-01-01 12:51:00,2016-01-01 13:51:00,US/Central,KORD,41.9875,-87.9319,Bensenville,DuPage,IL,60106.0,2016-01-01,12:51:00
386390,W-386468,Snow,Light,2016-01-03 21:51:00,2016-01-03 22:51:00,US/Central,KORD,41.9875,-87.9319,Bensenville,DuPage,IL,60106.0,2016-01-03,21:51:00
386391,W-386469,Snow,Light,2016-01-03 23:12:00,2016-01-04 04:51:00,US/Central,KORD,41.9875,-87.9319,Bensenville,DuPage,IL,60106.0,2016-01-03,23:12:00
386392,W-386470,Snow,Light,2016-01-04 14:51:00,2016-01-04 18:51:00,US/Central,KORD,41.9875,-87.9319,Bensenville,DuPage,IL,60106.0,2016-01-04,14:51:00


In [25]:
# dropping out columns that will not be used for evaluation

air_weather = common_airports.drop(['EventId', 'StartTime(UTC)', 'EndTime(UTC)', 'TimeZone', 'LocationLat', 'LocationLng', 'City', 'County', 'State', 'ZipCode'], axis=1)

In [26]:
air_weather.head()

,Type,Severity,AirportCode,WeatherDate,Time
386388,Snow,Light,KORD,2016-01-01,11:51:00
386389,Snow,Moderate,KORD,2016-01-01,12:51:00
386390,Snow,Light,KORD,2016-01-03,21:51:00
386391,Snow,Light,KORD,2016-01-03,23:12:00
386392,Snow,Light,KORD,2016-01-04,14:51:00


In [27]:
for i in air_weather.index:
    print(type(air_weather.WeatherDate[i]))

<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime

<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime

<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime

<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime

<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime

...

<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime

<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime

<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime

<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime

<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime

In [29]:
# Selecting relevant date ranges to apply to our selected dataset
from datetime import datetime

In [33]:
JAN_2018 = air_weather[(air_weather.WeatherDate > datetime(2018,1,1).date()) & 
                       (air_weather.WeatherDate <= datetime(2018,1,7).date())]

In [35]:
DEC_2018 = air_weather[(air_weather.WeatherDate > datetime(2018,12,24).date()) & 
                       (air_weather.WeatherDate <= datetime(2018,12,31).date())]

In [36]:
JAN_2019 = air_weather[(air_weather.WeatherDate > datetime(2019,1,1).date()) & 
                       (air_weather.WeatherDate <= datetime(2019,1,7).date())]

In [37]:
DEC_2019 = air_weather[(air_weather.WeatherDate > datetime(2019,12,24).date()) & 
                       (air_weather.WeatherDate <= datetime(2019,12,31).date())]

In [55]:
JAN_2020 = air_weather[(air_weather.WeatherDate > datetime(2020,1,1).date()) & 
                       (air_weather.WeatherDate <= datetime(2020,1,7).date())]

In [39]:
weather = pd.concat([JAN_2018, DEC_2018, JAN_2019, DEC_2019], axis=0)

In [40]:
weather.sort_values('WeatherDate')

,Type,Severity,AirportCode,WeatherDate,Time
2403696,Fog,Moderate,KDEN,2018-01-02,04:17:00
2403697,Fog,Moderate,KDEN,2018-01-02,09:53:00
3647467,Fog,Moderate,KLAX,2018-01-02,16:53:00
387695,Snow,Light,KORD,2018-01-03,08:51:00
5854908,Snow,Light,KIAH,2018-01-03,04:07:00
...,...,...,...,...,...
4193468,Rain,Light,KSEA,2019-12-31,23:05:00
4193467,Rain,Light,KSEA,2019-12-31,18:53:00
4193466,Rain,Light,KSEA,2019-12-31,11:45:00
4193465,Rain,Light,KSEA,2019-12-31,07:53:00


In [41]:
weather.Type.value_counts()

Rain             321
Fog               70
Snow              29
Storm              2
Precipitation      1
Name: Type, dtype: int64

In [42]:
weather.Severity.value_counts()

Light       270
Moderate     88
Severe       47
Heavy        17
UNK           1
Name: Severity, dtype: int64

In [43]:
# One-hot encoding for different types of weather events

weather['Snow'] = weather['Type'].apply(lambda x: 1 if x == 'Snow' else 0)
weather['Rain'] = weather['Type'].apply(lambda x: 1 if x == 'Rain' else 0)
weather['Fog'] = weather['Type'].apply(lambda x: 1 if x == 'Fog' else 0)
weather['Storm'] = weather['Type'].apply(lambda x: 1 if x == 'Storm' else 0)
weather['Precipitation'] = weather['Type'].apply(lambda x: 1 if x == 'Precipitation' else 0)
weather['Hail'] = weather['Type'].apply(lambda x: 1 if x == 'Hail' else 0)


# One-hot encoding for different severities of weather events

weather['Light'] = weather['Severity'].apply(lambda x: 1 if x == 'Light' else 0)
weather['Moderate'] = weather['Severity'].apply(lambda x: 1 if x == 'Moderate' else 0)
weather['Heavy'] = weather['Severity'].apply(lambda x: 1 if x == 'Heavy' else 0)
weather['Severe'] = weather['Severity'].apply(lambda x: 1 if x == 'Severe' else 0)
weather['UNK'] = weather['Severity'].apply(lambda x: 1 if x == 'UNK' else 0)
weather['Other'] = weather['Severity'].apply(lambda x: 1 if x == 'Other' else 0)

In [44]:
weather.head()

,Type,Severity,AirportCode,WeatherDate,Time,Snow,Rain,Fog,Storm,Precipitation,Hail,Light,Moderate,Heavy,Severe,UNK,Other
387695,Snow,Light,KORD,2018-01-03,08:51:00,1,0,0,0,0,0,1,0,0,0,0,0
387696,Snow,Light,KORD,2018-01-07,10:47:00,1,0,0,0,0,0,1,0,0,0,0,0
387697,Snow,Light,KORD,2018-01-07,18:51:00,1,0,0,0,0,0,1,0,0,0,0,0
387698,Rain,Light,KORD,2018-01-07,19:39:00,0,1,0,0,0,0,1,0,0,0,0,0
387699,Rain,Light,KORD,2018-01-07,20:40:00,0,1,0,0,0,0,1,0,0,0,0,0


In [45]:
weather = weather.drop(['Type', 'Severity'], axis=1)

In [46]:
weather.head()

,AirportCode,WeatherDate,Time,Snow,Rain,Fog,Storm,Precipitation,Hail,Light,Moderate,Heavy,Severe,UNK,Other
387695,KORD,2018-01-03,08:51:00,1,0,0,0,0,0,1,0,0,0,0,0
387696,KORD,2018-01-07,10:47:00,1,0,0,0,0,0,1,0,0,0,0,0
387697,KORD,2018-01-07,18:51:00,1,0,0,0,0,0,1,0,0,0,0,0
387698,KORD,2018-01-07,19:39:00,0,1,0,0,0,0,1,0,0,0,0,0
387699,KORD,2018-01-07,20:40:00,0,1,0,0,0,0,1,0,0,0,0,0


In [48]:
type(weather.Time[387695])

datetime.time

In [51]:
from datetime import time

weather['AM/PM'] = weather['Time'].apply(lambda y: 1 if y > time(12,0,0) else 0)

# setting 0 for AM and 1 for PM

In [69]:
# removing time column as it is no longer needed

In [59]:
weather = weather.drop('Time', axis=1)

In [60]:
weather.head()

,AirportCode,WeatherDate,Snow,Rain,Fog,Storm,Precipitation,Hail,Light,Moderate,Heavy,Severe,UNK,Other,AM/PM
387695,KORD,2018-01-03,1,0,0,0,0,0,1,0,0,0,0,0,0
387696,KORD,2018-01-07,1,0,0,0,0,0,1,0,0,0,0,0,0
387697,KORD,2018-01-07,1,0,0,0,0,0,1,0,0,0,0,0,1
387698,KORD,2018-01-07,0,1,0,0,0,0,1,0,0,0,0,0,1
387699,KORD,2018-01-07,0,1,0,0,0,0,1,0,0,0,0,0,1


In [56]:
# One-hot encoding for different types of weather events for test data

JAN_2020['Snow'] = JAN_2020['Type'].apply(lambda x: 1 if x == 'Snow' else 0)
JAN_2020['Rain'] = JAN_2020['Type'].apply(lambda x: 1 if x == 'Rain' else 0)
JAN_2020['Fog'] = JAN_2020['Type'].apply(lambda x: 1 if x == 'Fog' else 0)
JAN_2020['Storm'] = JAN_2020['Type'].apply(lambda x: 1 if x == 'Storm' else 0)
JAN_2020['Precipitation'] = JAN_2020['Type'].apply(lambda x: 1 if x == 'Precipitation' else 0)
JAN_2020['Hail'] = JAN_2020['Type'].apply(lambda x: 1 if x == 'Hail' else 0)


# One-hot encoding for different severities of weather events

JAN_2020['Light'] = JAN_2020['Severity'].apply(lambda x: 1 if x == 'Light' else 0)
JAN_2020['Moderate'] = JAN_2020['Severity'].apply(lambda x: 1 if x == 'Moderate' else 0)
JAN_2020['Heavy'] = JAN_2020['Severity'].apply(lambda x: 1 if x == 'Heavy' else 0)
JAN_2020['Severe'] = JAN_2020['Severity'].apply(lambda x: 1 if x == 'Severe' else 0)
JAN_2020['UNK'] = JAN_2020['Severity'].apply(lambda x: 1 if x == 'UNK' else 0)
JAN_2020['Other'] = JAN_2020['Severity'].apply(lambda x: 1 if x == 'Other' else 0)

JAN_2020['AM/PM'] = JAN_2020['Time'].apply(lambda y: 1 if y > time(12,0,0) else 0)

JAN_2020 = JAN_2020.drop(['Type', 'Severity', 'Time'], axis=1)

<ipython-input-56-7dead5b819da>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  JAN_2020['Snow'] = JAN_2020['Type'].apply(lambda x: 1 if x == 'Snow' else 0)
<ipython-input-56-7dead5b819da>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  JAN_2020['Rain'] = JAN_2020['Type'].apply(lambda x: 1 if x == 'Rain' else 0)
<ipython-input-56-7dead5b819da>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

In [57]:
JAN_2020.head()

,AirportCode,WeatherDate,Snow,Rain,Fog,Storm,Precipitation,Hail,Light,Moderate,Heavy,Severe,UNK,Other,AM/PM
389157,KORD,2020-01-04,1,0,0,0,0,0,1,0,0,0,0,0,0
2780782,KDFW,2020-01-03,0,0,1,0,0,0,0,1,0,0,0,0,0
2780783,KDFW,2020-01-03,0,0,1,0,0,0,0,0,0,1,0,0,0
2780784,KDFW,2020-01-03,0,0,1,0,0,0,0,0,0,1,0,0,0
3648152,KLAX,2020-01-03,0,0,1,0,0,0,0,1,0,0,0,0,0


In [70]:
# removing weather station 'K' coding from airport code for easier reference to original data
# taking care to only remove the first instance of 'K', incase there was a 'K' in the general airport code

In [62]:
weather.AirportCode = weather.AirportCode.apply(lambda z: z.replace('K', '', 1))

In [63]:
JAN_2020.AirportCode = JAN_2020.AirportCode.apply(lambda z: z.replace('K', '', 1))

In [71]:
# save files to csv to allow for loading into further notebooks

In [64]:
weather.to_csv("weather.csv", index=False)
JAN_2020.to_csv('jan2020.csv', index=False)